In [1]:
import numpy as np
import pandas as pd
import sys
sys.path.append("../")
sys.path.append("../..")

In [2]:
input_file = './data_miss_original.csv'
imputation_parameter ={
"imputation_method":[
    {
        "min":0,
        "max":1,
        "method":"mean"
    },
    {
        "min":2,
        "max":4,
        "method":"linear"
    },
    {
        "min":5,
        "max":10,
        "method":"brits"
    }
],
    "totalNanLimit":0.3
}

from KETIPrePartialDataPreprocessing.dataPreprocessing import dataIngestion
from KETIPrePartialDataPreprocessing.dataPreprocessing.partial_data_processing import partialDataProcessing

DI = dataIngestion.getData()
input_data = DI.getFileInput(input_file, 'timedate')
MDP = partialDataProcessing()
MDP.setData(input_data[:10000])
output_data = MDP.dataCleaning(imputation_parameter)

temp
-40 80
co2
0 99999
pm10
0 9999


NameError: name 'MissingPatternDetection' is not defined

In [ ]:
input_data = getData().getFileInput(input_file, 'timedate')
MDP = partialDataProcessing()
MDP.setData(input_data[:10000])
output_data = MDP.dataCleaning(imputation_parameter)


In [ ]:
output_data